In [97]:
from dataclasses import dataclass
from math import gcd, ceil
import re
import numpy as np
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools import cache
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest, batched, pairwise
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, data, quantify, atom, atoms, list_atoms, list_multiply, mapt, sortdict
from aocutils.grid import gridneigh, arr_to_dict, grid_to_dict, neighbors, complexneighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d, rotate, E, W, N, S, winds, TURNLEFT, TURNRIGHT
from aocutils.maze import graphparse, bfs, dijkstra, distances, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst, shoelace
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot, animate
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard

inf = float('inf')

In [98]:
set([1,2,3]) | set([1,25,3])

{1, 2, 3, 25}

In [141]:
f = open("in.txt").read().split("\n")
lines = [list(line) for line in f]

opp = {'G':'E', 'E':'G'}

def move(loc, wind):
    return (loc[0] + wind[0], loc[1] + wind[1])


def get_distances(loc, goal):
    creatures = {c.loc for c in players['E']} | {c.loc for c in players['G']}
    
    distances = {}
    tocheck = deque([loc])
    seen = {loc}
    dis = -1
    while tocheck:
        dis += 1
        for _ in range(len(tocheck)):
            cur = tocheck.popleft()
            if cur == goal:
                return distances
            distances[cur] = dis
            for n in [move(cur, wind) for wind in [E, S, N, W]]:
                if n == goal:
                    seen.add(n)
                    tocheck.append(n)                
                if n not in seen and n in points and n not in creatures:
                    seen.add(n)
                    tocheck.append(n)

    
    
    
    
def bfs(c):
    
    opponents = {p.loc for p in players[opp[c.creature]]}
    friends = {p.loc for p in players[c.creature]}
    print('     ', c, 'opponents', opponents)
    in_range = []
    distances = {}
    tocheck = deque([c.loc])
    seen = set(c.loc)
    dis = -1
    while tocheck:
        dis += 1
        for _ in range(len(tocheck)):
            loc = tocheck.popleft()
            distances[loc] = dis
            for n in [move(loc, wind) for wind in [E, S, N, W]]:
                if n not in seen and n in points:
                    seen.add(n)
                    if n in opponents:
                        if dis == 0:
                            return c.loc, 'in range'
                        in_range.append(n)
                    elif n not in friends:
                        tocheck.append(n)
    reachable = []
    print('      in_range_enemies', c, in_range)
    for loc in in_range:
        for n in [move(loc, wind) for wind in [E, S, N, W]]:
            if n in distances:
                reachable.append((distances[n], n))
    print('      reachable', reachable)
    if not reachable:
        return c.loc, 'no target'
    closest  = sorted(reachable)[0][1]
    target =  closest
    print('      target', target, 'self', c.loc)
    dis = get_distances(target, c.loc)
    print('      these are the distancestotarget', target, dis)
    paths = []
    if not dis:
        return c.loc, 'no target'
    for n in [move(c.loc, wind) for wind in [E, S, N, W]]:
        if n in dis:
            paths.append((dis[n], n))
    return sorted(paths)[0][1], 'move'
        


In [163]:


class Elf():
    def __init__(self, loc, creature,ap):
        self.loc = loc
        self.hp = 200
        self.ap = ap
        self.creature = creature
        
    def select_target(self):
        if self.hp <= 0:
            return
        next_loc, action = bfs(self)
        print('   ', self, next_loc, action)
        if action == 'in range':
            self.attack()
            
        if action == 'no target':
            return
            
        if action == 'move':
            print('i moved', next_loc)
            self.loc = next_loc
            self.attack()
    def attack(self):
        
        #select find best target
        print(self, 'attacking')
        options = []
        enemies = {e.loc: e for e in players[opp[self.creature]]}
        for n in [move(self.loc, wind) for wind in [E, S, N, W]]:
            if n in enemies:
                options.append((enemies[n].hp, enemies[n]))
        if not options:
            return
        options.sort()
        print('were hitting',options[0][1])
        options[0][1].get_hit(self.ap)
        
    def get_hit(self, ap):
        self.hp -= ap
        global hits
        if self.creature == 'E':
            hits.append(turns)
        if self.hp <= 0:
            print('I died', self.__repr__())
            players[self.creature].remove(self)
            print(len(players['E']), len(players['G']), 'remaining')
            if not len(players['E']) or not len(players['G']):
                print('no full round!!!!')
                sys.exit()
    
    def __repr__(self):
        return f'{self.creature} {self.loc} {self.hp} {self.ap}'
    
    def __lt__(self, other):
         return self.loc < other.loc
    
    
hits = []

players = defaultdict(set)
points = set()

for r,c, val in iterate(lines):
    if val in '.GE':
        points.add((r,c)) 
        if val in 'GE':
            if val == 'G':
                c = Elf((r,c), val,3)
            else:
                c = Elf((r,c), val,23)
            players[val].add(c)
turns = 0
orders =[]
import copy
while players['G'] and players['E']:
    order = [p for p in players['E']] + [p for p in players['G']]
    order.sort()
    orders.append(copy.deepcopy(order))
    print('new round', turns)
    for p in order:
        print(p)
    for player in order:
        player.select_target()
    turns += 1
    print('turn ended', turns)

print(len(players['E']))
# print(turns * (sum(p.hp for p in players['G']) + sum(p.hp for p in players['E'])))

# order = [p for p in players['E']] + [p for p in players['G']]
# order.sort()
# print('new round', turns)
# for p in order:
#     print(p)  

new round 0
G (1, 21) 200 3
G (6, 20) 200 3
G (7, 12) 200 3
G (8, 15) 200 3
G (8, 19) 200 3
G (9, 17) 200 3
G (9, 19) 200 3
G (11, 10) 200 3
G (12, 8) 200 3
G (14, 5) 200 3
G (15, 6) 200 3
G (16, 10) 200 3
E (16, 21) 200 23
G (18, 3) 200 3
E (18, 11) 200 23
G (19, 7) 200 3
E (21, 9) 200 23
E (21, 19) 200 23
G (22, 5) 200 3
G (22, 7) 200 3
E (22, 17) 200 23
E (22, 26) 200 23
E (23, 17) 200 23
G (24, 14) 200 3
G (25, 2) 200 3
E (25, 19) 200 23
G (27, 1) 200 3
G (27, 8) 200 3
E (28, 26) 200 23
E (30, 18) 200 23
      G (1, 21) 200 3 opponents {(23, 17), (25, 19), (28, 26), (21, 19), (21, 9), (22, 17), (22, 26), (16, 21), (30, 18), (18, 11)}
      in_range_enemies G (1, 21) 200 3 [(16, 21), (18, 11), (21, 9), (22, 17), (21, 19), (23, 17), (25, 19), (22, 26), (30, 18), (28, 26)]
      reachable [(48, (17, 21)), (16, (15, 21)), (30, (19, 11)), (26, (17, 11)), (28, (18, 10)), (35, (22, 9)), (31, (20, 9)), (33, (21, 8)), (40, (22, 18)), (38, (21, 17)), (43, (21, 20)), (41, (22, 19)), (45, (20,

NameError: name 'sys' is not defined

In [164]:
print(turns * (sum(p.hp for p in players['G']) + sum(p.hp for p in players['E'])))

order = [p for p in players['E']] + [p for p in players['G']]
order.sort()
print('new round', turns)
for p in order:
    print(p)  

52688
new round 37
E (10, 19) 158 23
E (11, 10) 179 23
E (11, 11) 29 23
E (11, 18) 80 23
E (12, 10) 200 23
E (13, 9) 185 23
E (14, 9) 32 23
E (15, 10) 161 23
E (17, 9) 200 23
E (18, 9) 200 23


In [127]:
turns

[]

In [102]:
(213692 - 215053 /83*82) / 3

410.0

In [103]:
for i, o in enumerate(orders):
    print(i, o)

0 [E (1, 2) 200 3, G (2, 5) 200 3, E (4, 1) 200 3, G (4, 3) 200 3, G (4, 5) 200 3, G (5, 5) 200 3]
1 [E (1, 3) 200 3, G (1, 5) 200 3, G (3, 5) 200 3, E (5, 1) 200 3, G (5, 3) 200 3, G (5, 5) 200 3]
2 [E (1, 4) 197 3, G (1, 5) 197 3, G (2, 5) 200 3, E (5, 2) 197 3, G (5, 3) 197 3, G (5, 5) 200 3]
3 [E (1, 4) 191 3, G (1, 5) 194 3, G (2, 4) 200 3, E (5, 2) 194 3, G (5, 3) 194 3, G (5, 5) 200 3]
4 [E (1, 4) 185 3, G (1, 5) 191 3, G (2, 4) 200 3, E (5, 2) 191 3, G (5, 3) 191 3, G (5, 5) 200 3]
5 [E (1, 4) 179 3, G (1, 5) 188 3, G (2, 4) 200 3, E (5, 2) 188 3, G (5, 3) 188 3, G (5, 5) 200 3]
6 [E (1, 4) 173 3, G (1, 5) 185 3, G (2, 4) 200 3, E (5, 2) 185 3, G (5, 3) 185 3, G (5, 5) 200 3]
7 [E (1, 4) 167 3, G (1, 5) 182 3, G (2, 4) 200 3, E (5, 2) 182 3, G (5, 3) 182 3, G (5, 5) 200 3]
8 [E (1, 4) 161 3, G (1, 5) 179 3, G (2, 4) 200 3, E (5, 2) 179 3, G (5, 3) 179 3, G (5, 5) 200 3]
9 [E (1, 4) 155 3, G (1, 5) 176 3, G (2, 4) 200 3, E (5, 2) 176 3, G (5, 3) 176 3, G (5, 5) 200 3]
10 [E (1, 

In [104]:
turns

55

In [105]:
# grid
grid, neigh = gridneigh("in.txt", to_dict=True, parser=lambda x: [(ch) for ch in x])


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (33,) + inhomogeneous part.

In [ ]:
# graph
conn = graphparse("in.txt", parser=lambda line: [x for x in line.split()])
conn = graphparse(
    "in.txt",
    parser=lambda line: [int(x) if x.isnumeric() else x for x in line.split()],
    weights=True,
)

In [7]:
E = 1; W = -1; N = 1j; S = -1j
TURNLEFT = lambda x: x * 1j
TURNRIGHT = lambda x: x * -1j

# parse grid with complex numbers
f = open("in.txt").read().split("\n")
lines = [list(line.split()) for line in f]
grid = arr_to_dict(lines, tocomplex=True)
complexneighbors(3+2j)

{(2+2j), (3+1j), (3+3j), (4+2j)}